In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/A_Needle_In_A_Data_Haystack

/content/drive/MyDrive/A_Needle_In_A_Data_Haystack


In [ ]:
!pip -q install transformers accelerate evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
TRAIN_CSV = "data/v2/train/combined_train.csv"
VAL_CSV   = "data/v2/val/combined_val.csv"
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LEN = 1024
SEED = 42
BATCH_SIZE = 8
GRAD_ACCUM = 2
EPOCHS = 2
LR = 3e-5

import os, math, numpy as np, pandas as pd, torch, random
from datasets import load_dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, TrainingArguments, Trainer)
from sklearn.metrics import mean_absolute_error, cohen_kappa_score, accuracy_score
import evaluate
import torch

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

In [ ]:
from huggingface_hub import login
import wandb
!hf auth login --token "YOUR_TOKEN"
!wandb login # your api key

PROJECT = "deberta-star-regression"
RUN_NAME = "deberta-v3-base_regression"

wandb.init(
    project=PROJECT,
    name=RUN_NAME,
    config={
        "model_name": "microsoft/deberta-v3-base",
        "task": "regression",
        "max_len": MAX_LEN,
        "lr": LR,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "grad_accum": GRAD_ACCUM,
        "seed": SEED,
    },
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `jetbrains` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `jetbrains`
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
raw = load_dataset(
    "csv",
    data_files={
        "train": TRAIN_CSV,
        "val": VAL_CSV
    }
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["review_data"],
        max_length=MAX_LEN,
    )


tokenized = raw.map(tokenize_fn, batched=True, remove_columns=[c for c in raw["train"].column_names if c not in ("labels")])


Map:   0%|          | 0/105000 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/10500 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1,
)
model.config.problem_type = "regression"

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
mae = evaluate.load("mae")
mse = evaluate.load("mse")
spearman = evaluate.load("spearmanr")

def regression_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.reshape(-1)  # shape (N,1) -> (N,)
    labels = labels.reshape(-1)

    mae_res = mae.compute(predictions=preds, references=labels)["mae"]
    rmse_res = np.sqrt(mse.compute(predictions=preds, references=labels)["mse"])
    r2 = r2_score(labels, preds)
    sp = spearman.compute(predictions=preds, references=labels)["spearmanr"]

    return {"mae": mae_res, "rmse": rmse_res, "r2": r2, "spearman": sp}



In [ ]:
OUTPUT_DIR = "/deberta-regression"

from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,

    logging_strategy="steps",
    logging_steps=100,
    report_to="none",

    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,

    save_strategy="no",
    save_total_limit=None,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=regression_metrics,
)


/tmp/ipython-input-864358559.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
train_result = trainer.train()
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Best checkpoint saved to:", OUTPUT_DIR)


Step,Training Loss
100,5.131200
200,0.661000
300,0.547500
400,0.492500
500,0.440700
600,0.397500
700,0.402000
800,0.382100
900,0.362400
1000,0.393200


Best checkpoint saved to: /deberta-regression


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

eval_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=64,
    dataloader_drop_last=False
)

trainer = Trainer(
    model=model,
    args=eval_args,
    tokenizer=tokenizer,
    compute_metrics=regression_metrics
)


/tmp/ipython-input-273157423.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from sklearn.metrics import r2_score


print("\nValidation metrics:")
val_metrics = trainer.evaluate(tokenized["val"])
for k, v in val_metrics.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")



Validation metrics:


eval_loss: 0.3304
eval_model_preparation_time: 0.0079
eval_mae: 0.3790
eval_rmse: 0.5748
eval_r2: 0.7675
eval_spearman: 0.7607
eval_runtime: 185.9444
eval_samples_per_second: 56.4680
eval_steps_per_second: 28.2340


In [ ]:
test_df = pd.read_csv("data/v2/test/books_test.csv")
tokenizer = AutoTokenizer.from_pretrained("deberta")
model = AutoModelForSequenceClassification.from_pretrained("deberta")
MAX_LEN = 512
BATCH_SIZE = 64

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

test_csv = "data/v2/test/electronics_test.csv"
model_dir = "deberta"

MAX_LEN = 512
BATCH_SIZE = 64

test_df = pd.read_csv(test_csv)

for col in ["predicted_rating", "rating_diff"]:
    if col not in test_df.columns:
        test_df[col] = np.nan

remaining_idx = np.where(test_df["predicted_rating"].isna())[0]
if len(remaining_idx) == 0:
    print("All rows already have predictions. Nothing to do.")
else:
    print(f"Rows to process: {len(remaining_idx)}")

tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, local_files_only=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

with torch.no_grad():
    for start in tqdm(range(0, len(remaining_idx), BATCH_SIZE), desc="Predicting"):
        batch_idx = remaining_idx[start:start+BATCH_SIZE]
        batch_texts = test_df.loc[batch_idx, "review_data"].tolist()

        enc = tokenizer(
            batch_texts,
            truncation=True,
            max_length=MAX_LEN,
            padding=True,
            return_tensors="pt"
        ).to(device)

        logits = model(**enc).logits.squeeze(-1).detach().cpu().numpy()

        test_df.loc[batch_idx, "predicted_rating"] = logits
        test_df.loc[batch_idx, "rating_diff"] = np.abs(
            logits - test_df.loc[batch_idx, "rating"].values
        )

test_df.to_csv(test_csv, index=False)
print(f"Saved predictions to {test_csv}")

print(test_df[["review_data", "predicted_rating", "rating_diff"]].head())


Rows to process: 467308


Predicting:   0%|          | 0/7302 [00:00<?, ?it/s]

Saved predictions to data/v2/test/electronics_test.csv
                                         review_data  predicted_rating  \
0  [CAT=Electronics] [TITLE=Items look real from ...          5.141633   
1  [CAT=Electronics] [TITLE=good product with qui...          5.160303   
2  [CAT=Electronics] [TITLE=Five Stars] I hope I ...          5.231755   
3  [CAT=Electronics] [TITLE=Excellent Camera and ...          5.216500   
4  [CAT=Electronics] [TITLE=Nontechnical person l...          5.213367   

   rating_diff  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
